In [1]:
import sys
sys.path.append('../ry')
from libry import *
from numpy import *

In [22]:
# load and display a configuration
K = Config()
K.addFile('../test/boxProblem.g')
D = K.view()
K.getFrameNames()

['base',
 'box',
 'boxBo',
 'boxLe',
 'boxBa',
 'ballR',
 'block',
 'target',
 'boxBa>target']

In [23]:
# create a standard path optimization problem with 4 phases, each of .2 seconds, and 10 steps
komo = K.komo_path(5., 10, .2)

In [24]:
# create tau-DOFs (time intervals) in the optimization problem and add respective constraints
komo.addTimeOptimization()

In [25]:
# take one object, and first 'switch' it to become freely movable without costs,
# then, starting at phase .7, make it dynamic (imposing NE equations)
obj = 'ballR'
#komo.addSwitch_magic(.0, 'base', obj)
komo.makeObjectsFree(["ballR"]);


In [26]:
# (old) a way to constrain the object position to be 'before' the box before phase 0.2
#komo.addObjective(time=[0., .2], type=OT.ineq, feature=FS.position, frames=[obj], scaleTrans=[[0.,1.,0.]], target=[1.])

In [27]:
# objectives for collision (permanent) and target (at phase-time 4.)
komo.addObjective(type=OT.sos, feature=FS.accumulatedCollisions)
komo.addObjective(time=[5.], type=OT.eq, feature=FS.distance, frames=['target', obj], scale=[3e1])

In [32]:
# simple structure of this solution: bounce on the bottom of the box at times 1, 2, and 3
komo.addSwitch_dynamicTrans(1., -1., 'base', obj)
komo.addInteraction_elasticBounce(2., "boxBo", obj, .8)
komo.addInteraction_elasticBounce(3., "boxBo", obj, .8)
komo.addInteraction_elasticBounce(4., "boxBo", obj, .8)

In [29]:
komo.optimize()

In [30]:
komoView = komo.view()

In [10]:
komo.displayTrajectory()
#(todo: use proper display thread, as for LGP optimizer)

In [11]:
komo.display()

In [15]:
# read out solution: the full frame path, the tau path (time optimization), list of interaction forces
obj_path = komo.getPathFrames([obj])
tau_path = komo.getPathTau()
forces = komo.getForceInteractions()
forces

[{'at': 9,
  'force': [1.3550911638451476e-08, 0.05844131299791279, -0.47950453006185223],
  'from': 'boxBo',
  'poa': [0.34012283450520486, -1.2649754081669191, 0.6496912259335912],
  'to': 'ballR'},
 {'at': 19,
  'force': [2.9973237755030936e-07,
   -4.200392864356983e-06,
   -0.37865886519765213],
  'from': 'boxBo',
  'poa': [0.25396826587662497, -0.3367204033048931, 0.649999243860761],
  'to': 'ballR'},
 {'at': 29,
  'force': [1.4412681122039584e-07,
   -3.2396513888794734e-06,
   -0.3181579885035238],
  'from': 'boxBo',
  'poa': [0.18138934740955584, 0.4518070300818377, 0.6499775453844219],
  'to': 'ballR'}]

Conversion from steps to phase and time:
```
phase=0 is the FIXED initial configuration and corresponds to step=-1
step=0 is the first configuration s.t. optimization
\tau(s) gives the time between step s-1 and step s
phase(step s) = (s+1)/stepsPerPhase
time(step s) = sum i=0:s tau(i)
```

In [33]:
D=0
komoView=0
K=0